# インポート

In [2]:
import pandas as pd
from pathlib import Path

COMMON_DATA_DIR = Path("..", "..", "common", "data")
RAWDF_DIR = COMMON_DATA_DIR / "rawdf"

# データ加工

## レース結果テーブルの加工

In [33]:
df = pd.read_csv(RAWDF_DIR / "results.csv", sep="\t")

In [37]:
df.iloc[:, 0:10]

,race_id,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差
0,202301010101,1,5,5,サトミノキラリ,牡2,55.0,横山武史,1:09.5,NaN
1,202301010101,2,8,8,ベアゴーゴー,牝2,55.0,浜中俊,1:09.5,クビ
2,202301010101,3,6,6,ハピアーザンエバー,牡2,55.0,藤岡佑介,1:10.0,2.1/2
3,202301010101,4,4,4,デビルシズカチャン,牝2,55.0,ルメール,1:10.2,1.1/2
4,202301010101,5,1,1,ウィスピースノー,牝2,55.0,吉田隼人,1:10.3,1/2
...,...,...,...,...,...,...,...,...,...,...
47667,202310030812,8,1,1,リヴァーレ,牝3,53.0,太宰啓介,2:42.9,1.1/4
47668,202310030812,9,6,7,シュアーウィナー,牡5,55.0,川端海翼,2:43.5,3.1/2
47669,202310030812,10,2,2,アウローラシエル,牡3,55.0,和田竜二,2:43.6,3/4
47670,202310030812,11,4,4,ハイグッドエース,牡4,55.0,西塚洸二,2:43.7,3/4


In [35]:
df["tansho_odds"] = df["単勝"].astype(float)

ValueError: could not convert string to float: '---'

In [36]:
df["tansho_odds"] = pd.to_numeric(df["tansho_odds"], errors="coerce")

KeyError: 'tansho_odds'

In [5]:
df["着順"]

0         1
1         2
2         3
3         4
4         5
         ..
47667     8
47668     9
47669    10
47670    11
47671    12
Name: 着順, Length: 47672, dtype: object

In [6]:
df["rank"] = pd.to_numeric(df["着順"], errors="coerce") # errors="coerce"で変換できない文字を欠損値に変換.
df["rank"].value_counts(dropna=False)

rank
3.0     3460
1.0     3459
2.0     3456
4.0     3456
5.0     3456
6.0     3443
7.0     3424
8.0     3367
9.0     3259
10.0    3101
11.0    2895
12.0    2637
13.0    2342
14.0    2047
15.0    1708
16.0    1279
NaN      399
17.0     280
18.0     204
Name: count, dtype: int64

In [7]:
df.dropna(subset=["rank"], inplace=True) # 欠損値を省く

In [8]:
df["rank"].value_counts(dropna=False)

rank
3.0     3460
1.0     3459
2.0     3456
4.0     3456
5.0     3456
6.0     3443
7.0     3424
8.0     3367
9.0     3259
10.0    3101
11.0    2895
12.0    2637
13.0    2342
14.0    2047
15.0    1708
16.0    1279
17.0     280
18.0     204
Name: count, dtype: int64

In [9]:
df["馬番"].astype(int)

0        5
1        8
2        6
3        4
4        1
        ..
47667    1
47668    7
47669    2
47670    4
47671    5
Name: 馬番, Length: 47273, dtype: int64

In [10]:
df["性齢"].str[0].value_counts()

性齢
牡    25038
牝    19922
セ     2313
Name: count, dtype: int64

ラベルエンコーディング：カテゴリ変数を整数にマッピングするデータ加工手法

In [11]:
sex_mapping = {"牡": 0, "牝": 1, "セ": 2}

In [12]:
df["性齢"].str[0].map(sex_mapping).value_counts()

性齢
0    25038
1    19922
2     2313
Name: count, dtype: int64

In [13]:
df["性齢"].str[1:].astype(int)

0        2
1        2
2        2
3        2
4        2
        ..
47667    3
47668    5
47669    3
47670    4
47671    3
Name: 性齢, Length: 47273, dtype: int64

In [50]:
df["斤量"]

0        55.0
1        55.0
2        55.0
3        55.0
4        55.0
         ... 
47667    53.0
47668    55.0
47669    55.0
47670    55.0
47671    52.0
Name: 斤量, Length: 47273, dtype: float64

In [15]:
df["単勝"] = pd.to_numeric(df["単勝"], errors="coerce")

In [17]:
df["単勝"]

0          1.2
1          4.1
2         59.9
3         16.6
4         23.9
         ...  
47667     62.8
47668    125.6
47669      9.8
47670     37.2
47671      6.2
Name: 単勝, Length: 47273, dtype: float64

In [18]:
df["人気"].astype(int)

0         1
1         2
2         6
3         3
4         5
         ..
47667    11
47668    12
47669     5
47670    10
47671     4
Name: 人気, Length: 47273, dtype: int64

In [25]:
df["weight"] = df["馬体重"].str.extract(r"(\d+)").astype(int) # .str.extractを用いた正規表現で抜き出す.
df["weight"]

0        452
1        454
2        438
3        450
4        434
        ... 
47667    394
47668    470
47669    496
47670    440
47671    442
Name: weight, Length: 47273, dtype: int64

In [31]:
df["weight_diff"] = df["馬体重"].str.extract(r"\((.+)\)").astype(int) # エスケープの()と抜き出す()を使って増減を抜き出す.

In [32]:
df["weight_diff"]

0        -4
1         2
2        -6
3         2
4       -10
         ..
47667     2
47668    -4
47669     8
47670    -2
47671     6
Name: weight_diff, Length: 47273, dtype: int64

In [ ]:
df["tansho_odds"] = df["単勝"].astype(float)

In [39]:
COMMON_DATA_DIR = Path("..", "..", "common", "data")
INPUT_DIR = COMMON_DATA_DIR / "rawdf"
MAPPING_DIR = COMMON_DATA_DIR / "mapping"
OUTPUT_DIR = Path("..", "data", "01_preprocessed")
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

In [41]:
df = pd.read_csv(INPUT_DIR / "results.csv", sep="\t")

In [42]:
df.columns

Index(['race_id', '着順', '枠番', '馬番', '馬名', '性齢', '斤量', '騎手', 'タイム', '着差',
       'ﾀｲﾑ指数', '通過', '上り', '単勝', '人気', '馬体重', '調教ﾀｲﾑ', '厩舎ｺﾒﾝﾄ', '備考', '調教師',
       '馬主', '賞金(万円)', 'horse_id', 'jockey_id', 'trainer_id', 'owner_id'],
      dtype='object')

In [43]:
df["rank"] = pd.to_numeric(df["着順"], errors="coerce")

In [44]:
df.dropna(subset=["rank"], inplace=True)

In [45]:
df["rank"] = df["rank"].astype(int)

In [46]:
df["sex"] = df["性齢"].str[0].map(sex_mapping)

In [49]:
df["impost"] = df["片量"].astype(float)

KeyError: '片量'

In [51]:
df = pd.read_csv(RAWDF_DIR / "horse_results.csv", sep="\t")
df

,horse_id,日付,開催,天気,R,レース名,映像,頭数,枠番,馬番,...,着差,ﾀｲﾑ指数,通過,ペース,上り,馬体重,厩舎ｺﾒﾝﾄ,備考,勝ち馬(2着馬),賞金
0,2011106610,2024/04/29,高知,雨,8.0,りさママ生誕記念特別(C3),NaN,10,6.0,7,...,4.7,**,2-4-6-7,0.0-38.1,41.9,477(-1),NaN,NaN,イモータルスモーク,NaN
1,2011106610,2024/04/14,高知,晴,8.0,C3ー15,NaN,11,7.0,8,...,2.3,**,2-2-2-3,0.0-40.0,42.2,478(-1),NaN,NaN,カイラシ,NaN
2,2011106610,2024/03/27,高知,晴,3.0,C3ー10,NaN,12,3.0,3,...,1.1,**,2-2-2-2,0.0-40.6,41.6,479(+3),NaN,NaN,デルマジゾウ,6.0
3,2011106610,2024/03/20,高知,小雨,2.0,C3ー7,NaN,11,7.0,9,...,0.7,**,7-8-5-6,0.0-41.5,41.9,476(0),NaN,NaN,ヤマニンバシリーサ,6.0
4,2011106610,2024/03/06,高知,曇,3.0,伊予馬事畜産特別(C3),NaN,11,6.0,7,...,1.4,**,3-3-3-4,0.0-38.4,39.7,476(-2),NaN,NaN,ララプロフォン,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211453,2021110161,2024/03/31,2阪神4,曇,9.0,バイオレットS(OP),NaN,11,4.0,4,...,1.2,**,9-9,34.2-38.1,38.1,478(0),NaN,NaN,エートラックス,NaN
211454,2021110161,2024/02/03,1東京3,晴,4.0,3歳1勝クラス,NaN,8,7.0,7,...,-0.1,**,2-2,37.2-36.9,36.8,478(-2),NaN,NaN,(スタンリーテソーロ),780.0
211455,2021110161,2023/12/10,5中山4,晴,7.0,2歳1勝クラス,NaN,16,4.0,7,...,2.2,**,7-7-6-6,37.1-37.9,39.5,480(0),NaN,NaN,ミッキーファイト,NaN
211456,2021110161,2023/11/18,5東京5,晴,1.0,2歳未勝利,NaN,15,6.0,12,...,-0.4,**,3-2,34.4-38.3,38.2,480(-4),NaN,NaN,(ミルトパワー),550.0
